In [6]:
import pandas as pd
import os

# List of CSV file names
csv_files = [
    r"C:\Users\hmane\Desktop\label_studio_images\twitter_data\memes\x2018_batch_300_label_studio_input.csv",
    r"C:\Users\hmane\Desktop\label_studio_images\twitter_data\memes\x2019_batch_300_label_studio_input.csv",
    r"C:\Users\hmane\Desktop\label_studio_images\twitter_data\memes\x2020_batch_300_label_studio_input.csv",
    r"C:\Users\hmane\Desktop\label_studio_images\twitter_data\memes\x2021_batch_300_label_studio_input.csv",
    r"C:\Users\hmane\Desktop\label_studio_images\twitter_data\memes\x2022_batch_300_label_studio_input.csv",
    r"C:\Users\hmane\Desktop\label_studio_images\twitter_data\memes\x2023_batch_300_label_studio_input.csv"
]

def update_url(url):
    return url.replace("https://github.com", "https://raw.githubusercontent.com").replace("/tree", "")

for csv_file in csv_files:
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Update the 'image' column
    df['image'] = df['image'].apply(update_url)
    
    # Save the updated CSV file
    df.to_csv(csv_file, index=False)

print("URLs have been updated successfully.")


URLs have been updated successfully.


In [29]:
import os
import pandas as pd
from collections import defaultdict

# Paths to the directories
main_folder = r"C:\Users\hmane\Desktop\label_studio_images"
subfolders_1_to_16 = [f'batch_{i}' for i in range(1, 17)]
subfolders_17_to_50 = [f'batch_{i}' for i in range(17, 51)]
subfolders = subfolders_1_to_16 + [os.path.join('10k_batches', subfolder) for subfolder in subfolders_17_to_50]

# Read the CSV file
csv_file_path = r"C:\Users\hmane\Desktop\folders\memes\ct_data\ig_fb_combined_race_lgbtq_01012016_07312023.csv"
df = pd.read_csv(csv_file_path)

# Extract post IDs from URLs
df['post_id'] = df['URL'].apply(lambda x: x.split("/")[-1])

# Initialize a list to store the matched data
matched_data = []

# Iterate through the batch folders
for subfolder in subfolders:
    folder_path = os.path.join(main_folder, subfolder)
    for image_name in os.listdir(folder_path):
        # Extract the post ID from the image name
        if '-' in image_name:
            post_id = image_name.split('-')[1].split('.')[0]
        else:
            post_id, _ = os.path.splitext(image_name)  # Assuming the image names have extensions
        matching_row = df[df['post_id'] == post_id]
        if not matching_row.empty:
            url = matching_row['URL'].values[0]
            post_created_date = matching_row['Post Created Date'].values[0]
            matched_data.append([url, image_name, post_created_date, subfolder])

# Create a DataFrame for the matched data
matched_df = pd.DataFrame(matched_data, columns=['URL', 'Image Name', 'Post Created Date', 'Batch Name'])

# Add the year to matched_df
matched_df['year'] = pd.to_datetime(matched_df['Post Created Date']).dt.year

# Initialize a dictionary to count images per year
images_per_year = defaultdict(int)

# Iterate through the DataFrame to count images per year
for index, row in matched_df.iterrows():
    post_year = row['year']
    images_per_year[post_year] += 1

# Convert the dictionary to a DataFrame for easy visualization
images_per_year_df = pd.DataFrame(list(images_per_year.items()), columns=['Year', 'Image Count'])

# Save the image count per year to a new CSV file
output_csv_path = r"C:\Users\hmane\Desktop\image_count_per_year.csv"
images_per_year_df.to_csv(output_csv_path, index=False)

print(f"Image count per year has been saved to {output_csv_path}")

# # Optionally, save the matched_df with the year and batch name columns to a new CSV file
# matched_df_output_path = r"C:\Users\hmane\Desktop\matched_data_with_year_and_batch.csv"
# matched_df.to_csv(matched_df_output_path, index=False)

# print(f"Matched DataFrame with year and batch name has been saved to {matched_df_output_path}")


c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,25,30,34,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Image count per year has been saved to C:\Users\hmane\Desktop\image_count_per_year.csv


In [19]:
from collections import defaultdict

# Initialize a dictionary to count images per year
images_per_year = defaultdict(int)

# Iterate through the DataFrame to count images per year
for index, row in matched_df.iterrows():
    post_created_date = row['Post Created Date']
    post_year = pd.to_datetime(post_created_date).year
    row['year']=post_year
    images_per_year[post_year] += 1

# Convert the dictionary to a DataFrame for easy visualization
images_per_year_df = pd.DataFrame(list(images_per_year.items()), columns=['Year', 'Image Count'])

# Save the image count per year to a new CSV file
output_csv_path = 'path_to_output_csv_file.csv'
images_per_year_df.to_csv(output_csv_path, index=False)

print(f"Image count per year has been saved to {output_csv_path}")

Image count per year has been saved to path_to_output_csv_file.csv


In [ ]:
images_per_year_df

In [28]:
df[df['URL']=='https://www.facebook.com/409876055752592/posts/2418712421535602']

,Unnamed: 0,Unnamed: 0.1,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,...,Sponsor Name,Sponsor Category,Total Interactions (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x ),Overperforming Score,Platform,Like and View Counts Disabled,Photo,Title,Total Interactions (weighted — Likes 1x Comments 1x ),post_id
1153045,1153045,5405,Hoosiers for Common Sense PAC,Indianatalks,1.000640e+14,POLITICAL_ORGANIZATION,US,Hoosiers for Common Sense PAC,12/2/2012 2:35,9233.0,...,NaN,NaN,455,15.17,FB,NaN,NaN,NaN,NaN,2418712421535602


In [26]:
matched_df['year'] = pd.to_datetime(matched_df['Post Created Date']).dt.year
matched_df[matched_df.year==2019]['URL'].tolist()

['https://www.facebook.com/409876055752592/posts/2418712421535602',
 'https://www.facebook.com/431507603634165/posts/2616379508480286',
 'https://www.facebook.com/42822870949/posts/10157414011335950',
 'https://www.facebook.com/468691496624810/posts/1358582780969006',
 'https://www.facebook.com/725359304229585/posts/2432305773534921',
 'https://www.facebook.com/1646874365530472/posts/2576069095944323',
 'https://www.facebook.com/1426945007395508/posts/2589898327766831',
 'https://www.facebook.com/134956789923521/posts/2552766878142488',
 'https://www.facebook.com/298759552462/posts/10157806872437463',
 'https://www.facebook.com/600184653336065/posts/2766198263401349',
 'https://www.facebook.com/2258677777532687/posts/2711966588870468',
 'https://www.facebook.com/969859323108916/posts/2704488989645932',
 'https://www.facebook.com/5975759948/posts/10151355317254949',
 'https://www.facebook.com/1470552979894321/posts/2532961023653506',
 'https://www.facebook.com/177123619000633/posts/2706

In [6]:
matched_df['Post Created Date'].min()

'1/1/2020'

In [7]:
matched_df['Post Created Date'].max()

'9/7/2018'

Create Batches

In [1]:
import os
import csv

csv_input_file = r"C:\Users\hmane\Desktop\folders\memes\ct_data\ig_fb_combined_race_lgbtq_01012016_07312023.csv"
base_output_file = 'CT_annotations_batch_{}-{}.csv'
image_base_folder_path = r"C:\Users\hmane\Desktop\label_studio_images\10k_batches\"  # This is the directory containing batch folders
github_base_url = "https://raw.githubusercontent.com/heranmane/label_studio_images/main/10k_batches"


In [8]:
import pandas as pd
df = pd.read_csv(csv_input_file)

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,25,30,34,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
import os
import csv

# Define paths and variables
csv_input_file = r"C:\Users\hmane\Desktop\folders\memes\ct_data\ig_fb_combined_race_lgbtq_01012016_07312023.csv"
image_base_folder_path = r"C:/Users/hmane/Desktop/label_studio_images/images_yr4/"  # This is the directory containing batch folders
base_output_file = 'CT_annotations_batch_{}.csv'
github_base_url = "https://raw.githubusercontent.com/heranmane/label_studio_images/main/10k_batches/"
# Read original CSV to get the text associated with images
text_data = {}
with open(csv_input_file, 'r', newline='', encoding='utf-8') as infile:
    csv_reader = csv.DictReader(infile)
    for row in csv_reader:
        post_url = row['URL']
        post_id = post_url.split("/")[-1]
        text_data[post_id] = row['Message']

# Process batches
batch_folders = [f for f in os.listdir(image_base_folder_path) if f.startswith('batch_')]
batch_folders.sort(key=lambda x: int(x.split('_')[1]))  # Sort folders numerically by batch number

# Create one CSV per batch
for batch in batch_folders:
    batch_path = os.path.join(image_base_folder_path, batch)
    output_csv_file = base_output_file.format(batch)

    with open(output_csv_file, 'w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['batch', 'image', 'text']
        csv_writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        csv_writer.writeheader()

        for image_filename in os.listdir(batch_path):
            # Ensure the image filename follows the expected format
            parts = image_filename.split('-')
            if len(parts) > 1:
                post_id = parts[1].split('.')[0]
                if post_id in text_data:
                    image_url = github_base_url + os.path.join(batch, image_filename).replace('\\', '/')
                    new_row = {
                        'batch': batch,
                        'image': image_url,
                        'text': text_data[post_id]
                    }
                    csv_writer.writerow(new_row)

    print(f"CSV file created for batch {batch}: {output_csv_file}")

CSV file created for batch batch_17: CT_annotations_batch_batch_17.csv
CSV file created for batch batch_18: CT_annotations_batch_batch_18.csv
CSV file created for batch batch_19: CT_annotations_batch_batch_19.csv
CSV file created for batch batch_20: CT_annotations_batch_batch_20.csv
CSV file created for batch batch_21: CT_annotations_batch_batch_21.csv
CSV file created for batch batch_22: CT_annotations_batch_batch_22.csv
CSV file created for batch batch_23: CT_annotations_batch_batch_23.csv
CSV file created for batch batch_24: CT_annotations_batch_batch_24.csv
CSV file created for batch batch_25: CT_annotations_batch_batch_25.csv
CSV file created for batch batch_26: CT_annotations_batch_batch_26.csv
CSV file created for batch batch_27: CT_annotations_batch_batch_27.csv
CSV file created for batch batch_28: CT_annotations_batch_batch_28.csv
CSV file created for batch batch_29: CT_annotations_batch_batch_29.csv
CSV file created for batch batch_30: CT_annotations_batch_batch_30.csv
CSV fi

In [26]:
# import os

# def get_post_ids_and_image_names(base_path):
#     post_ids = {}
#     image_names = {}
#     batch_folders = []

#     # Iterate through all subfolders in the base path
#     for root, dirs, files in os.walk(base_path):
#         batch_name = os.path.basename(root)
#         if files:
#             batch_folders.append(batch_name)
#             for file in files:
#                 if file not in image_names:
#                     image_names[file] = batch_name
#                 else:
#                     image_names[file] += f", {batch_name}"
                
#                 parts = file.split('-')
#                 if len(parts) > 1:
#                     post_id = parts[1].split('.')[0]
#                     if post_id not in post_ids:
#                         post_ids[post_id] = batch_name
#                     else:
#                         post_ids[post_id] += f", {batch_name}"
    
#     return post_ids, image_names, batch_folders

# # Paths to the main folders
# folder1 = r"C:/Users/hmane/Desktop/label_studio_images/10k_batches/"
# folder2 = r"C:/Users/hmane/Desktop/label_studio_images/split_images/ct_week1/"

# # Get post_ids and image_names from both folders
# post_ids1, image_names1, batch_folders1 = get_post_ids_and_image_names(folder1)
# post_ids2, image_names2, batch_folders2 = get_post_ids_and_image_names(folder2)

# # # Print all batch folder names
# # print("Batch folders in folder1:")
# # for batch in batch_folders1:
# #     print(batch)

# # print("\nBatch folders in folder2:")
# # for batch in batch_folders2:
# #     print(batch)

# # Find duplicates
# duplicate_post_ids = set(post_ids1.keys()).intersection(set(post_ids2.keys()))
# duplicate_image_names = set(image_names1.keys()).intersection(set(image_names2.keys()))

# # Output results
# print(f"\nDuplicate post IDs: {len(duplicate_post_ids)} found.")
# for post_id in duplicate_post_ids:
#     print(f"{post_id} found in batches: {post_ids1[post_id]} and {post_ids2[post_id]}")

# print(f"\nDuplicate image names: {len(duplicate_image_names)} found.")
# for image_name in duplicate_image_names:
#     print(f"{image_name} found in batches: {image_names1[image_name]} and {image_names2[image_name]}")


In [27]:
import os
import csv

def count_csv_rows(directory):
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    for csv_file in csv_files:
        file_path = os.path.join(directory, csv_file)
        with open(file_path, 'r', newline='', encoding='utf-8') as infile:
            csv_reader = csv.reader(infile)
            row_count = sum(1 for row in csv_reader) - 1  # Subtract 1 to exclude the header
            print(f"CSV file: {csv_file}, Number of rows: {row_count}")

# Paths to the directories containing CSV files
folder1 = r"C:/Users/hmane/Desktop/label_studio_images/10k_batches/"
folder2 = r"C:/Users/hmane/Desktop/label_studio_images/split_images/ct_week1/"

# Print CSV file names and row counts in folder1
print("CSV files in folder1:")
count_csv_rows(folder1)

# Print CSV file names and row counts in folder2
print("\nCSV files in folder2:")
count_csv_rows(folder2)


CSV files in folder1:
CSV file: CT_annotations_batch_batch_17.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_18.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_19.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_20.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_21.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_22.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_23.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_24.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_25.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_26.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_27.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_28.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_29.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_30.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_31.csv, Number of rows: 300
CSV

In [28]:
# pd.read_csv(r'C:\Users\hmane\Desktop\label_studio_images\split_images\CT_annotations_batch_batch_47.csv')

create tracking sheet

In [17]:
import csv

# List of CSV files uploaded for annotation
csv_files = [
    "CT_annotations_batch_1-3.csv",
    "CT_annotations_batch_4-6.csv",
    "CT_annotations_batch_7-9.csv",
    "CT_annotations_batch_10-12.csv",
    "CT_annotations_batch_13-15.csv",
    "CT_annotations_batch_16.csv"
]

# Annotators information (example names/IDs)
annotators = [("Annotator1", "Annotator2", "TieBreaker")]

# Create the tracking CSV
tracking_csv = "annotation_tracking.csv"
with open(tracking_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["CSV File Name", "Annotator 1", "Annotator 1 Status", 
                     "Annotator 2", "Annotator 2 Status", "Tie Breaker", 
                     "Tie Breaker Status", "Overall Status"])
    
    # Write initial data
    for csv_file in csv_files:
        for annotator1, annotator2, tie_breaker in annotators:
            # Initial statuses can be adjusted as needed
            writer.writerow([csv_file, annotator1, "Not Started", annotator2, 
                             "Not Started", tie_breaker, "Not Started", "Incomplete"])

print("Tracking CSV created:", tracking_csv)


Tracking CSV created: annotation_tracking.csv


In [14]:
import pandas as pd 
df = pd.read_csv(r'C:\Users\hmane\Downloads\split_images\CT_annotations_batch_batch_1-batch_3.csv')
df

,batch,image,text
0,batch_1,https://raw.githubusercontent.com/heranmane/Me...,BLM Roundup & Ended Up at a Kill Lot in Texas!...
1,batch_1,https://raw.githubusercontent.com/heranmane/Me...,".Cherokee Speedway, we’re coming for ya! 👊 The..."
2,batch_1,https://raw.githubusercontent.com/heranmane/Me...,Marsha “Pay it No Mind” Johnson (1945-1992) wa...
3,batch_1,https://raw.githubusercontent.com/heranmane/Me...,🚨Taco Tuesday 🚨🌮🦐🌮 🚨2X1 Tacos with a Drink pur...
4,batch_1,https://raw.githubusercontent.com/heranmane/Me...,"Jacks Fam, please join us in welcoming Raegen ..."
...,...,...,...
295,batch_3,https://raw.githubusercontent.com/heranmane/Me...,Chili Cornbread Casserole https://ed.gr/duszu:...
296,batch_3,https://raw.githubusercontent.com/heranmane/Me...,HIGH WIND WARNING THRU 7 AM: A storm complex i...
297,batch_3,https://raw.githubusercontent.com/heranmane/Me...,The weather team is tracking a winter storm sy...
298,batch_3,https://raw.githubusercontent.com/heranmane/Me...,✨ NEW PRODUCT ALERT! ✨ We need your vote becau...


In [7]:
df2 = pd.read_csv(r'C:\Users\hmane\Downloads\split_images\CT_annotations_batch_batch_3-batch_5.csv')
df2

,batch,image,text
0,batch_3,https://raw.githubusercontent.com/heranmane/Me...,City officials expect an 80% chance of rain on...
1,batch_3,https://raw.githubusercontent.com/heranmane/Me...,Reposted from @prparadenyc We are currently pl...
2,batch_3,https://raw.githubusercontent.com/heranmane/Me...,"""We seek the history of Jewish women out of ou..."
3,batch_3,https://raw.githubusercontent.com/heranmane/Me...,The big guy got Caught.
4,batch_3,https://raw.githubusercontent.com/heranmane/Me...,On this special day honoring the 100th birthda...
...,...,...,...
295,batch_5,https://raw.githubusercontent.com/heranmane/Me...,Now introducing Iwate Prefecture! Iwate is kno...
296,batch_5,https://raw.githubusercontent.com/heranmane/Me...,Ever wondered about different Eggplants? Here'...
297,batch_5,https://raw.githubusercontent.com/heranmane/Me...,NaN
298,batch_5,https://raw.githubusercontent.com/heranmane/Me...,"Hey Folks! On Sunday, January 9 at 8pm ET, I w..."
